## EyeWatchOne Engineering Prototype

Hyper local weather related accident prediction.

Gives a browser/mobile based heatmap overlay on satellite images showing where accidents are predicted to occur given historical accidents and incoming weather.


Targeted towards government planning for road emergency services, but can also be used for insurance claims adjustment, logistics companies's route planning, and much more. 

##### This notebook can be run on windows, osx, linux, any platform with aws-cli installed.

### 1. Fix errors in AWS command line interface

I had an issue with windows not associating .py with my Python install.

fix .py file association error
http://superuser.com/questions/429604/passing-arguments-to-a-python-script-file-association-not-found-windows-7-on-i

### 2. Upload test insurance data to S3 bucket: eyewatchone

code adapted from http://boto.cloudhackers.com/en/latest/s3_tut.html

Insurance data:  
470 MB zipped csv  
from https://www.kaggle.com/c/ClaimPredictionChallenge/

In [ ]:
! aws s3 cp data/example_compressed_entry.zip s3://eyewatchone --grants \
    read=uri=http://acs.amazonaws.com/groups/global/AllUsers full=emailaddress=hollisnolan@gmail.com

In [ ]:
! aws s3 cp data/dictionary.html s3://eyewatchone --grants \
    read=uri=http://acs.amazonaws.com/groups/global/AllUsers full=emailaddress=hollisnolan@gmail.com

#### Move the full sets of Insurance Data to S3

In [ ]:
! aws s3 cp data/test_set.zip s3://eyewatchone --grants \
    read=uri=http://acs.amazonaws.com/groups/global/AllUsers full=emailaddress=hollisnolan@gmail.com

In [ ]:
! aws s3 cp data/train_set.zip s3://eyewatchone --grants \
    read=uri=http://acs.amazonaws.com/groups/global/AllUsers full=emailaddress=hollisnolan@gmail.com

In [ ]:
! aws s3 ls s3://eyewatchone

### 3. Spin up analytics EC2 instance

(right click for paste in windows)

Create an image from the Data Science Tookbox ami

In [ ]:
# create an instance, DS-toolbox ami and XL size

!aws ec2 run-instances --image-id ami-d1737bb8 --count 1 --instance-type m3.xlarge --key-name .boto --security-groups my-sg

Historical weather data:  
20 GB EBS snapshot  
https://aws.amazon.com/datasets/daily-global-weather-measurements-1929-2009-ncdc-gsod/  

In [ ]:
# attach EBS historical weather data 

ec2-attach-volume snap-ac47f4c5 --instance i-582bf5dc --device  /dev/sdf

or retstart a stopped instance

In [ ]:
# restart a stopped instance 

!aws ec2 start-instances --instance-ids i-582bf5dc --region us-east-1

In [ ]:
# get PublicDnsName and IP
!aws ec2 describe-instances --instance-ids i-582bf5dc --region us-east-1 

Ssh into the instance and start a notebook

In [ ]:
# ssh in (in terminal) 
ssh -X -i ~/.ssh/aws.pem ubuntu@ec2-54-88-161-196.compute-1.amazonaws.com

# update 
sudo apt-get update && sudo apt-get upgrade

### super update (in terminal, very long)
sudo do-release-upgrade

# update pip 
sudo pip install pip --upgrade

# install notebook
sudo pip install --upgrade "ipython[notebook]"

# open ports 
In the Security Group, select Inbound, then Edit, and then Add Rule. The Port Range should be 8888 and the Sourceis 0.0.0.0/0

# start tmux 
tmux new -s notebook

# new notebook with ip specified
jupyter notebook --no-browser --ip=0.0.0.0

# grab ip address of instance and navigate to 
http://54.88.161.196:8888/tree
        
# save this notebook, close it locally, and use the "upload" button to move it into the aws instance
# rerun notebook from there

In [ ]:
! scp -i /users/hn/.ssh/aws.pem 09-Demo.ipynb ubuntu@ec2-54-88-161-196.compute-1.amazonaws.com:~

### 4. Check historical weather in EBS public data set

how to 
http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ebs-attaching-volume.html

public weather
https://aws.amazon.com/datasets/daily-global-weather-measurements-1929-2009-ncdc-gsod/

In [ ]:
! pwd

List disks 

In [ ]:
! lsblk

mount disk 
(help from)
https://help.ubuntu.com/community/InstallingANewHardDrive

In [ ]:
!sudo mount /dev/xvdf /hist_weather/

In [ ]:
# install aws command line interface 
!sudo pip install awscli --upgrade

In [ ]:
# add credentials (in command line, not notebook)
!aws configure 

###  5. Move historical insurance to EBS

In [ ]:
!aws s3 ls s3://eyewatchone

In [ ]:
# make a directory for analytics 
!sudo mkdir eyewatchone

In [ ]:
# copy files from s3
!sudo aws s3 cp s3://eyewatchone/example_compressed_entry.zip eyewatchone/

In [ ]:
# unzip
!sudo unzip eyewatchone/example_compressed_entry.zip

###  6. Move a sample image to EC2 analytics instance 

Histoical satellite images:  
very very large, S3  
1 GB / image, updated daily 

Commands to explore files, what PRE means 
http://docs.aws.amazon.com/cli/latest/reference/s3/ls.html

In [ ]:
! pwd

In [ ]:
! 

In [ ]:
!aws s3 ls s3://nasanex/Landsat/gls/2000/001/012 --recursive 

In [ ]:
!aws s3 cp s3://nasanex/Landsat/gls/2000/001/012/p001r012_7x20010613.tar.gz .

In [ ]:
! tar -xvzf p001r012_7x20010613.tar.gz

In [ ]:
! rm p001r012_7x20010613.tar.gz

In [ ]:
import matplotlib.pyplot as plt
image = plt.imread('p001r012_7dt20010613_z24_10.tif')
image

Landsat specialized python utility 
http://landsat-util.readthedocs.org/en/latest/installation.html#ubuntu-14-10


In [ ]:
# install depencencies 
!sudo apt-get -y install python-pip python-numpy python-scipy \
libgdal-dev libatlas-base-dev gfortran libfreetype6-dev

In [ ]:
# install utility  ( reccomended at command line, long )
!yes | sudo pip install landsat-util

Alternate images source 
https://aws.amazon.com/datasets/ccafs-climate-data/?tag=datasets%23keywords%23climate  
6 TB, S3    
    

In [ ]:
! aws s3 ls s3://cgiardata/ccafs/ccafs-climate/data/eta/eta_south_america/baseline/1970s/hadcm_high/20min/ --recursive

### 7. Query Weather API, Store to EBS

code adapted from http://stackoverflow.com/questions/12965203/how-to-get-json-from-webpage-into-python-script
and https://www.wunderground.com/weather/api/d/docs?MR=1

In [ ]:
# create an instance, DS-toolbox ami and XL size
!aws ec2 run-instances --image-id ami-d1737bb8 --count 1 --instance-type m3.micro --key-name .boto --security-groups my-sg

In [ ]:
# start instance 
!aws ec2 start-instances --instance-ids i-bb95183f --region us-east-1 

In [ ]:
# get PublicDnsName
!aws ec2 describe-instances --instance-ids i-bb95183f --region us-east-1 

In [ ]:
# ssh in (in terminal) 
ssh -X -i ~/.ssh/aws.pem ubuntu@ec2-54-164-113-126.compute-1.amazonaws.com

# new notebook with ip specified
jupyter notebook --no-browser --ip=0.0.0.0

# grab ip address of instance and navigate to 
http://52.91.161.73:8888/tree
        
# move notebook to instance

Here we are going to use Docker to quickly get Kafka working 
https://github.com/tobegit3hub/standalone-kafka

In [ ]:
!sudo apt-get install -y docker

In [ ]:
! docker run -d --net=host -e HOSTNAME=localhost tobegit3hub/standalone-kafka

In [ ]:
! cd kafka

In [ ]:
! bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic weather_stream

In [ ]:
# open ports 
In the Security Group, select Inbound, then Edit, and then Add Rule, the Port Range should be 2181 and the Source is 0.0.0.0/0

In [ ]:
! bin/kafka-console-producer.sh --broker-list localhost:9092 --topic weather_stream

In [ ]:
! bin/kafka-console-consumer.sh --zookeeper localhost:2181 --topic weather_stream --from-beginning

In [ ]:
import urllib, json
from pprint import pprint

url = "http://api.wunderground.com/api/  api key   /conditions/q/CA/San_Francisco.json"
response = urllib.urlopen(url)
data = json.loads(response.read())
pprint(data)

### 8. Produce a Prediction

In [ ]:
import numpy as np

pred = np.random.rand(1)
pred

### 9. Tag the Image, Send to Team

In [ ]:
tagged = zip(pred, image)
tagged.write('image_tagged')

In [ ]:
! aws s3 cp image_tagged s3://eyewatchone --grants \
    read=uri=http://acs.amazonaws.com/groups/global/AllUsers full=emailaddress=hollisnolan@gmail.com

### 10. Clone the Analytics EC2, Scale for Production

In [ ]:
# create an AMI from the EC2 analytics image 
aws ec2 create-image --instance-id i-582bf5dc --name "eyewatchone-analytics" --description \
"AMI for EyeWatchOne's prediction and analytics EC2 XL or higher reccomended."

### 11. Stop the instances

In [ ]:
# stop analytics instance 
!aws ec2 stop-instances --instance-id i-582bf5dc --region us-east-1

In [ ]:
# stop streaming instance 
!aws ec2 stop-instances --instance-id i-bb95183f --region us-east-1

##### Always double check 

https://console.aws.amazon.com/ec2/v2/home?region=us-east-1#Instances:sort=instanceId